In [ ]:
import os
import glob
import cv2
import json
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry.multipolygon import MultiPolygon
from PIL import Image

In [ ]:
LABEL_PATH = "./labels/"
IMAGE_PATH = "./images/"
MASK_PATH = "./masks/mask/"
IMAGE_TRAIN_PATH = "./timages/img"

In [ ]:
labels = glob.glob(LABEL_PATH + "*pre_disaster.json")

In [ ]:
if not os.path.isdir(MASK_PATH):
    os.makedirs(MASK_PATH)
if not os.path.isdir(IMAGE_TRAIN_PATH):
    os.makedirs(IMAGE_TRAIN_PATH)

In [ ]:
def read_json(fname):
    with open(fname) as json_file:
        data_json = json.load(json_file)
        return data_json

In [ ]:
def check_building(img, elements):
    try:
        polygon = wkt.loads(elements)
        x, y = polygon.exterior.coords.xy

        bbox = {
            'x_min': round(min(x.tolist())), 
            'x_max': round(max(x.tolist())), 
            'y_min': round(min(y.tolist())), 
            'y_max': round(max(y.tolist()))
        }


        cut_image = Image.fromarray(img[bbox['y_min']:bbox['y_max'], bbox['x_min']:bbox['x_max']])

        avg_color_per_row = np.average(cut_image, axis=0)
        avg_color = np.average(avg_color_per_row, axis=0)

        if avg_color[0] > 10 and avg_color[1] > 10 and avg_color[2] > 10: 
            return True
        else:
            return False
    except:
        return False

In [ ]:
def create_mask(fdata):
    imgname = fdata['metadata']['img_name']
    img = cv2.imread(IMAGE_PATH + imgname)
    mask = np.zeros(list(PDIM))
    for cord in fdata['features']['xy']:
        elements = wkt.loads(cord['wkt'])
        build = check_building(img, cord['wkt'])
        if build:
            x, y = elements.exterior.coords.xy
            points = np.array(list(zip(x, y)))
            mask = cv2.fillPoly(mask, np.int32([points]), color=(255))
    cv2.imwrite(IMAGE_TRAIN_PATH + imgname, img)
    cv2.imwrite(MASK_PATH + imgname, mask)

In [ ]:
def create_training_images(files):
    for image in files:
        data = read_json(image)
        create_mask(data)

In [ ]:
create_training_images(labels)